# Train Traffic Lights Agents

Utilise les fonctions de @Binetruy

- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integre l'environnement pour RLlib et execute la simulation


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks.IssyOSMNetwork import IssyOSMNetwork
from flow.core.params import InFlows
from collections import OrderedDict
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import VehicleParams, SumoCarFollowingParams

/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages

## Importation du network d'Issy

On vérifie si IssyOSMNetwork est bien importé.

In [2]:
from flow.networks.IssyOSMNetwork import ADDITIONAL_NET_PARAMS, EDGES_DISTRIBUTION

print(ADDITIONAL_NET_PARAMS)
print(EDGES_DISTRIBUTION)

{'speed_limit': 50}
['-100822066', '4794817', '4783299#0', '155558218']


## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [3]:
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}), num_vehicles=10)
vehicles.add("rl",acceleration_controller=(RLController, {}), num_vehicles=2)

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [4]:
inflow = InFlows()

inflow.add(veh_type      = "human",
           edge          = "4794817",
           probability   = 0.3, 
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type      = "human",
           edge          = "4783299#0",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

inflow.add(veh_type       = "human",
           edge           = "-100822066",
           probability    = 0.25,
           depart_speed   = 7,
           depart_lane    = "random")

inflow.add(veh_type       = "rl",
           edge           = "-100822066",
           probability    = 0.05,
           depart_speed   = 7,
           depart_lane    = "random",
           color          = "blue")

inflow.add(veh_type       = "human",
           edge          = "155558218",
           probability   = 0.2,
           depart_speed  = 7,
           depart_lane   = "random")

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [5]:
from flow.envs.IssyEnv import IssyEnv1

## Lance une simulation avec Training RLlib

Pour qu'un environnement puisse être entrainé, l'environnement doit être accessible via l'importation à partir de flow.envs. 


<font color='red'>
Copier l'environnement créé dans un fichier .py et on importe l'environnement dans `flow.envs.__init__.py`.
Mettre le chemin absolu du fichier .osm .
</font> 

In [6]:
# possibles actions
action_spec = OrderedDict({ "30677963": [ "GGGGrrrGGGG", "rrrrGGGrrrr"],
                            "30763263": ["GGGGGGGGGG",  "rrrrrrrrrr"],
                            "30677810": [ "GGrr", "rrGG"]})

In [7]:
horizon  = 1000
SIM_STEP = 0.1
n_veh    = 12
rollouts = 10
n_cpus   = 3
discount_rate = 0.999

In [8]:
# SUMO PARAM
sumo_params = SumoParams(sim_step=SIM_STEP, render=False, restart_instance=True)

# ENVIRONMENT PARAM
ADDITIONAL_ENV_PARAMS = {"beta": n_veh, "action_spec": action_spec, "algorithm": "DQN", "tl_constraint_min": 100,  "tl_constraint_max": 600, "sim_step": SIM_STEP}
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=horizon, warmup_steps=1)

# NETWORK PARAM
path_file  = '/home/lino/Documents/DQN_CIL4SYS/DQN_CIL4SYS/notebooks/issy.osm'
net_params = NetParams(inflows=inflow, osm_path=path_file) 

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION)


flow_params = dict( exp_tag   = "ISSY_traffic", 
                    env_name  = IssyEnv1,  
                    network   = IssyOSMNetwork,
                    simulator = 'traci',
                    sim       = sumo_params,
                    env       = env_params,
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config)

# Setup RLlib library

Configures RLlib DQN algorithm to be used to train the RL model.

In [9]:
def setup_DQN_exp():

    alg_run   = 'DQN'
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config['num_workers']      = n_cpus
    config['train_batch_size'] = horizon * rollouts
    config['gamma']            = discount_rate
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config['horizon']          = horizon
    config["hiddens"]          = [256]
    config['model'].update({'fcnet_hiddens': [32, 32]})

    # save the flow params for replay
    flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config

Configures RLlib PPO algorithm to be used to train the RL model.

See: https://ray.readthedocs.io/en/latest/rllib-algorithms.html#proximal-policy-optimization-ppo

In [10]:
def setup_PPO_exp():

    alg_run   = 'PPO'
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config['num_workers']      = n_cpus
    config['train_batch_size'] = horizon * rollouts
    config['gamma']            = discount_rate
    config['use_gae']          = True
    config['lambda']           = 0.97
    config['kl_target']        = 0.02
    config['num_sgd_iter']     = 10
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config['horizon']          = horizon
    config['model'].update({'fcnet_hiddens': [32, 32]})

    # save the flow params for replay
    flow_json = json.dumps(flow_params,cls=FlowParamsEncoder,sort_keys=True,indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params,version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config

# Run Experiment

In [11]:
alg_run, gym_name, config = setup_DQN_exp()

ray.init(num_cpus=n_cpus + 1)

2020-04-14 15:40:06,519	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_15-40-06_518906_10943/logs.
2020-04-14 15:40:06,631	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:26528 to respond...
2020-04-14 15:40:06,750	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:20306 to respond...
2020-04-14 15:40:06,755	INFO services.py:809 -- Starting Redis shard with 2.93 GB max memory.
2020-04-14 15:40:06,783	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_15-40-06_518906_10943/logs.
2020-04-14 15:40:06,785	INFO services.py:1475 -- Starting the Plasma object store with 4.4 GB memory using /dev/shm.


{'node_ip_address': '192.168.0.48',
 'redis_address': '192.168.0.48:26528',
 'object_store_address': '/tmp/ray/session_2020-04-14_15-40-06_518906_10943/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-14_15-40-06_518906_10943/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-04-14_15-40-06_518906_10943'}

In [ ]:
exp_tag = {"run": alg_run,
           "env": gym_name,
           "config": {**config},
           "checkpoint_freq": 20,
           "checkpoint_at_end": True,
           "max_failures": 999,
           "stop": {"training_iteration": 1000}}


trials = run_experiments({flow_params["exp_tag"]: exp_tag})

2020-04-14 15:40:06,972	INFO trial_runner.py:176 -- Starting a new experiment.
2020-04-14 15:40:07,012	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-04-14 15:40:07,033	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 3.5/14.7 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 3.5/14.7 GB
Result logdir: /home/lino/ray_results/ISSY_traffic
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - DQN_IssyEnv1-v0_0:	RUNNING

(pid=10978) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10978)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=10978) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(p

(pid=10979) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=10979) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=10979) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=10979) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=10980) Warning: Discarding unknown comp

(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=10980) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=10977) Warning: Removed a road without junctions: -21549451
(pid=10977) Warning: Removed a road without junctions: 21549451
(pid=10977) Warning: The traffic light '30624291' does not control any links; it will not be build.
(pid=10977) Warning: Could not build program '0' for traffic light '30624291'
(pid=10977) Warning: Minor green from edge '-352962858#0' to ed

(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.0' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.0' to match departure speed.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.1' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.1' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.2' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.0' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.1' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.3' to match departure speed.
(pid=10979) Warning: Vehicle 'human_3' performs emergency braking with decel=-135.01 wished=4.50 severity=29.00, time=10.20.
(pid=10979) Warning: Vehicle 'human_2' performs emergency b

...

Result for DQN_IssyEnv1-v0_0:
  custom_metrics: {}
  date: 2020-04-14_15-59-24
  done: false
  episode_len_mean: 911.6666666666666
  episode_reward_max: 23.909999999999982
  episode_reward_mean: 5.966166666666659
  episode_reward_min: -15.152999999999954
  episodes_this_iter: 2
  episodes_total: 6
  experiment_id: b3ea59ed448945129499dc5e0ad92044
  hostname: lino-iMac
  info:
    grad_time_ms: 1172.887
    learner:
      default_policy:
        cur_lr: 0.0005000000237487257
        max_q: 0.9865819811820984
        mean_q: 0.08346664160490036
        mean_td_error: -0.014274551533162594
        min_q: -0.2908143401145935
        model: {}
    max_exploration: 0.5060800000000001
    min_exploration: 0.5060800000000001
    num_steps_sampled: 6048
    num_steps_trained: 4200000
    num_target_updates: 12
    opt_peak_throughput: 8525.968
    opt_samples: 10000.0
    replay_time_ms: 1404.662
    sample_time_ms: 301.996
    update_time_ms: 5.669
  iterations_since_restore: 6
  node_ip: 192.

(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.13' to match departure speed.
(pid=10977) Warning: Vehicle 'flow_40.1' performs emergency braking with decel=-9.13 wished=4.50 severity=1.03, time=45.90.
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=10980) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=10980) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=10980) Warning: Removed a road without junctions: -21549451
(pid=10980) Warning: Removed a road without junctions: 21549451
(pid=10980) Wa

(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.23' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.13' to match departure speed.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.6' to match departure speed.
(pid=10979) Warning: Vehicle 'flow_20.10' performs emergency braking with decel=-66.25 wished=4.50 severity=13.72, time=80.00.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.15' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.24' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.16' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.17' to match departure speed.
(pid=10979) Warning: Vehicle 'rl_1' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=85.90.
(pid=10980) Warning: Vehicle 'flow

(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.9' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.1' to match departure speed.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.10' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.2' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.5' to match departure speed.
(pid=10980) Warning: Vehicle 'flow_20.20' performs emergency braking with decel=-18.01 wished=4.50 severity=3.00, time=72.90.
(pid=10980) Warning: Vehicle 'flow_10.4' performs emergency braking with decel=-16.17 wished=4.50 severity=2.59, time=72.90.
(pid=10980) Warning: Vehicle 'flow_10.3' performs emergency braking with decel=-22.43 wished=4.50 severity=3.98, time=72.90.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.11' to match departure speed.
(pid=10979

(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.13' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.14' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.15' to match departure speed.
(pid=10979) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.13' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.16' to match departure speed.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.3' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.17' to match departure speed.
(pid=10980) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.4' to match departure speed.
(pid=10977) Warning: Choosing new speed factor 1.26 for vehicle 'flow_40.18' to match departure speed.
Result for DQN_IssyEnv1-v0_0:
  custom_metrics: {}
  date: 2020-04-14_16-18